In [3]:
# ==========================================
# 1. 환경 설정 및 경로 지정
# ==========================================
import sys
import os
from google.colab import drive

drive.mount('/content/drive')

# 본인의 경로에 맞게 수정
PROJECT_PATH = '/content/drive/MyDrive/DL_Project'
os.chdir(PROJECT_PATH)
sys.path.append(PROJECT_PATH)

# 라이브러리 임포트 (TimeDropout 추가)
try:
    from common.np import *
    from common.time_layers import TimeEmbedding, TimeLSTM, TimeDropout # TimeDropout 추가
    from common.layers import Affine, SoftmaxWithLoss
    from common.optimizer import Adam
    from utils import extract_utterances
    import pandas as pd
    from collections import Counter
    print("✅ 라이브러리 로드 성공. 사용 모듈:", np.__name__)
except ImportError as e:
    print(f"❌ 라이브러리 로드 실패: {e}")

# ==========================================
# 2. 데이터 전처리 및 로더 (기존과 동일)
# ==========================================
def load_vocab(train_csv, max_vocab=4000):
    df = pd.read_csv(train_csv)
    words = []
    print("⏳ Vocab 생성 중...")

    for _, row in df.iterrows():
        try:
            utts = extract_utterances(row['filename'], ['CHI'])
            text = " ".join([u.clean_text for u in utts]).lower()
            words.extend(text.split())
        except:
            continue

    counter = Counter(words)
    most_common = counter.most_common(max_vocab)

    vocab = {'<pad>': 0, '<unk>': 1}
    for i, (w, _) in enumerate(most_common):
        vocab[w] = i + 2

    print(f"✅ Vocab 크기: {len(vocab)}")
    return vocab

def load_data(csv_path, vocab, max_len=300):
    df = pd.read_csv(csv_path)
    x_data = []
    t_data = []
    label_map = {'TD': 0, 'SLI': 1}
    print(f"📂 데이터 로딩: {csv_path}")

    for _, row in df.iterrows():
        try:
            utts = extract_utterances(row['filename'], ['CHI'])
            if not utts: continue
            text = " ".join([u.clean_text for u in utts]).lower()
            tokens = text.split()
            ids = [vocab.get(w, vocab['<unk>']) for w in tokens]

            if len(ids) > max_len:
                ids = ids[:max_len]
            padded_ids = np.zeros(max_len, dtype=int)
            length = min(len(ids), max_len)
            padded_ids[:length] = ids[:length]

            x_data.append(padded_ids)
            t_data.append(label_map[row['group']])
        except:
            pass
    return np.array(x_data), np.array(t_data)

# ==========================================
# 3. 모델 정의 (Dropout 적용)
# ==========================================
class SLPClassifier:
    def __init__(self, vocab_size, wordvec_size, hidden_size, class_size=2, dropout_ratio=0.5):
        V, D, H, C = vocab_size, wordvec_size, hidden_size, class_size
        rn = np.random.randn

        # 가중치 초기화
        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H, C) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(C).astype('f')

        # 계층 생성: LSTM 뒤에 Dropout 추가
        self.layers = [
            TimeEmbedding(embed_W),
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False),
            TimeDropout(dropout_ratio) # Dropout 추가
        ]
        self.affine_layer = Affine(affine_W, affine_b)
        self.loss_layer = SoftmaxWithLoss()

        # 파라미터 수집
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
        self.params += self.affine_layer.params
        self.grads += self.affine_layer.grads

    def predict(self, xs):
        for layer in self.layers:
            xs = layer.forward(xs)

        # xs shape: (N, T, H) (Dropout 적용된 상태)
        # 마지막 시점의 은닉 상태 추출
        h_last = xs[:, -1, :]
        score = self.affine_layer.forward(h_last)
        return score

    def forward(self, xs, ts):
        score = self.predict(xs)
        loss = self.loss_layer.forward(score, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        dout = self.affine_layer.backward(dout)

        N, H = dout.shape
        # TimeEmbedding(layer[0])을 통해 T 추론
        T = len(self.layers[0].layers)

        dhs = np.zeros((N, T, H), dtype='f')
        dhs[:, -1, :] = dout

        for layer in reversed(self.layers):
            dhs = layer.backward(dhs)
        return dhs

    # 학습/추론 모드 전환 (Dropout 제어용)
    def set_train_flg(self, train_flg=True):
        for layer in self.layers:
            if hasattr(layer, 'train_flg'):
                layer.train_flg = train_flg

# ==========================================
# 4. 데이터 로드
# ==========================================
train_csv = 'split/gillam_train.csv'
dev_csv = 'split/gillam_dev.csv'
test_csv = 'split/gillam_test.csv'

vocab = load_vocab(train_csv, max_vocab=4000)
vocab_size = len(vocab)

x_train, t_train = load_data(train_csv, vocab)
x_dev, t_dev = load_data(dev_csv, vocab)
x_test, t_test = load_data(test_csv, vocab)

if np.__name__ == 'cupy':
    x_train, t_train = np.asarray(x_train), np.asarray(t_train)
    x_dev, t_dev = np.asarray(x_dev), np.asarray(t_dev)
    x_test, t_test = np.asarray(x_test), np.asarray(t_test)

print(f"Train Shape: {x_train.shape}")

# ==========================================
# 5. 학습 (Train Loop 수정)
# ==========================================
max_epoch = 15 # Dropout을 쓰면 학습이 더 오래 걸릴 수 있으므로 Epoch을 조금 늘림
batch_size = 16
hidden_size = 128
wordvec_size = 100
dropout_ratio = 0.5 # 드롭아웃 비율 설정 (보통 0.5)
learning_rate = 0.001

model = SLPClassifier(vocab_size, wordvec_size, hidden_size, dropout_ratio=dropout_ratio)
optimizer = Adam(lr=learning_rate)

data_size = len(x_train)
max_iters = data_size // batch_size

print(f"\n🚀 학습 시작 (Device: {np.__name__})")

for epoch in range(max_epoch):
    # [중요] 학습 모드로 설정 (Dropout 활성화)
    model.set_train_flg(True)

    idx = np.random.permutation(data_size)
    x_train = x_train[idx]
    t_train = t_train[idx]

    total_loss = 0
    loss_count = 0

    for i in range(max_iters):
        batch_x = x_train[i*batch_size : (i+1)*batch_size]
        batch_t = t_train[i*batch_size : (i+1)*batch_size]

        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)

        total_loss += loss
        loss_count += 1

    # [중요] 평가 모드로 설정 (Dropout 비활성화)
    model.set_train_flg(False)

    # 검증 (Dev Set)
    model_score = model.predict(x_dev)
    pred = np.argmax(model_score, axis=1)
    acc = np.sum(pred == t_dev) / len(t_dev)

    avg_loss = total_loss / loss_count
    print(f"| Epoch {epoch+1:02} | Loss {avg_loss:.4f} | Dev Acc {acc:.4f}")

# ==========================================
# 6. 최종 테스트
# ==========================================
# 테스트 전에도 평가 모드 확인
model.set_train_flg(False)

test_score = model.predict(x_test)
test_pred = np.argmax(test_score, axis=1)
test_acc = np.sum(test_pred == t_test) / len(t_test)

print(f"\n🏆 Final Test Accuracy: {test_acc*100:.2f}%")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 라이브러리 로드 성공. 사용 모듈: numpy
⏳ Vocab 생성 중...
✅ Vocab 크기: 3702
📂 데이터 로딩: split/gillam_train.csv
📂 데이터 로딩: split/gillam_dev.csv
📂 데이터 로딩: split/gillam_test.csv
Train Shape: (540, 300)

🚀 학습 시작 (Device: numpy)
| Epoch 01 | Loss 0.6145 | Dev Acc 0.7353
| Epoch 02 | Loss 0.5442 | Dev Acc 0.7353
| Epoch 03 | Loss 0.4928 | Dev Acc 0.7353
| Epoch 04 | Loss 0.4333 | Dev Acc 0.7059
| Epoch 05 | Loss 0.3652 | Dev Acc 0.6765
| Epoch 06 | Loss 0.3401 | Dev Acc 0.7059
| Epoch 07 | Loss 0.3030 | Dev Acc 0.7206
| Epoch 08 | Loss 0.3094 | Dev Acc 0.7206
| Epoch 09 | Loss 0.2774 | Dev Acc 0.7059
| Epoch 10 | Loss 0.2732 | Dev Acc 0.7353
| Epoch 11 | Loss 0.2747 | Dev Acc 0.7206
| Epoch 12 | Loss 0.2558 | Dev Acc 0.7353
| Epoch 13 | Loss 0.2486 | Dev Acc 0.7500
| Epoch 14 | Loss 0.2468 | Dev Acc 0.7500
| Epoch 15 | Loss 0.2630 | Dev Acc 0.7500

🏆 Final Test Accuracy: 72.06%
